In [5]:
import pandas as pd
import numpy as np
import zipfile

In [8]:
with zipfile.ZipFile('child-mind-institute-problematic-internet-use.zip') as z:
    with z.open('series_train.parquet/id=00f332d1/part-0.parquet') as f:
        #df = pd.read_csv(f)
        df_obs = pd.read_parquet(f)

df_obs    # time series data for one of the observations

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,-0.051997,0.011325,-0.991420,0.026643,-86.152954,0.0,30.000000,4185.000000,57240000000000,1,2,14.0
1,1,-0.008801,-0.036264,-1.000591,0.001868,-87.851646,0.0,30.000000,4185.000000,57245000000000,1,2,14.0
2,2,-0.008904,-0.035101,-1.000357,0.001776,-87.853790,0.0,30.333334,4185.000000,57250000000000,1,2,14.0
3,3,-0.009110,-0.035204,-1.000435,0.001675,-87.850815,0.0,30.666666,4185.000000,57255000000000,1,2,14.0
4,4,-0.009020,-0.035204,-1.000709,0.001846,-87.850815,0.0,31.000000,4185.000000,57260000000000,1,2,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
414379,414379,0.015927,-0.021750,-0.985415,0.000000,-88.438774,1.0,0.500000,3098.833252,55535000000000,4,2,38.0
414380,414380,0.016043,-0.021879,-0.985102,0.000000,-88.445999,1.0,0.500000,3098.666748,55540000000000,4,2,38.0
414381,414381,0.015888,-0.021814,-0.985428,0.000000,-88.451225,1.0,0.500000,3098.500000,55545000000000,4,2,38.0
414382,414382,0.016352,-0.021762,-0.985741,0.000000,-88.456245,1.0,0.500000,3098.333252,55550000000000,4,2,38.0


In [6]:
df_desc = pd.read_csv('content/data_dictionary.csv')

In [7]:
df = pd.read_csv('content/train.csv')

# Removing columns in which more than half the rows are null

In [ ]:
filter_null = df.isnull().sum().sort_values(ascending = True) <= (len(df) / 2)
filter_null = filter_null[filter_null == True]
list_of_columns_less_null = filter_null.index.to_list()
df_filter_null = df[list_of_columns_less_null]
df_filter_null.isna().sum().sum() / len(df_filter_null)

23.568434343434344

In [ ]:
len(df_filter_null)

3960

# Removing the outliers

In [ ]:
columns_df = df_filter_null.columns[1:]

In [ ]:
# Make a copy of the original DataFrame
df_removed_outliers = df_filter_null.copy()

# Initialize a mask of True values for all rows (no outliers initially)
mask = pd.Series(True, index=df_filter_null.index)

# Iterate over each column in columns_df
for column in columns_df:
    # Check if the column is numeric
    if pd.api.types.is_numeric_dtype(df_filter_null[column]):
        # Calculate Q1, Q3, and IQR for the column
        q1 = df_filter_null[column].quantile(0.25)
        q3 = df_filter_null[column].quantile(0.75)
        IQR = q3 - q1
        lower_bound = q1 - 1.5 * IQR
        upper_bound = q3 + 1.5 * IQR

        # Update the mask: keep rows within bounds or where values are NaN
        mask &= ((df_filter_null[column] >= lower_bound) & (df_filter_null[column] <= upper_bound)) | df_filter_null[column].isna()

# Apply the mask to filter rows, retaining the original values
df_removed_outliers = df_filter_null[mask]


In [ ]:
df_removed_outliers

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,PreInt_EduHx-Season,FGC-Season,Physical-Season,PreInt_EduHx-computerinternet_hoursday,Physical-Weight,Physical-Height,...,BIA-BIA_LST,BIA-BIA_LDM,BIA-BIA_ICW,BIA-BIA_Frame_num,BIA-BIA_Fat,BIA-BIA_FMI,BIA-BIA_FFMI,BIA-BIA_BMR,BIA-BIA_ECW,BIA-BIA_DEE
3,00115b9f,Winter,9,0,Winter,Summer,Summer,0.0,81.6,56.0,...,58.9338,16.7790,30.4041,2.0,18.82430,4.22033,14.0740,1131.43,15.5925,1923.44
4,0016bb22,Spring,18,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0069fbed,Summer,15,0,Summer,Spring,Spring,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0083e397,Summer,19,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,0087dd65,Spring,11,1,Spring,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,8,0,Fall,Fall,Fall,2.0,67.2,52.5,...,49.3301,15.8150,25.7118,1.0,14.66690,3.74130,13.4004,1035.27,11.0063,1759.96
3955,ff8a2de4,Fall,13,0,Fall,Fall,Fall,1.0,82.4,59.5,...,66.2889,17.9797,33.3709,1.0,11.58830,2.30138,14.0629,1206.88,19.4611,2051.70
3957,ffcd4dbd,Fall,11,0,Fall,Winter,Winter,0.0,109.8,60.0,...,71.3903,21.3403,33.9805,2.0,33.99670,6.63952,14.8043,1253.74,20.4825,2005.99
3958,ffed1dd5,Spring,13,0,Spring,Spring,Winter,1.0,87.0,70.7,...,86.2475,25.0054,41.3715,2.0,-5.90917,-0.83117,13.0684,1414.34,26.5323,2970.12


# Impute Na values with random values from the same column

In [ ]:
df_impute_random = df_removed_outliers.copy()
for column in columns_df:
  s = df_impute_random[column]

  # Step 1: Find the number of missing child heights for that gender.
  num_null = s.isna().sum()



  # Step 2: Sample num_null observed child heights for that gender.
  fill_values = np.random.choice(s.dropna(), num_null)

  # Step 3: Fill in missing
  s[s.isna()] = fill_values
  df_impute_random[column] = s





<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[s.isna()] = fill_values
<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[s.isna()] = fill_values
<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[s.isna()] = fill_values
<ipython-input-27-f464eed2bdc1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
df_impute_random



0

# Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_min_max = df_impute_random.copy()


In [ ]:
numeric_columns = df_impute_random.select_dtypes(include=['number']).columns.tolist()


In [ ]:
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_min_max[numeric_columns]), columns=numeric_columns)
df_min_max.loc[:, numeric_columns] = scaler.fit_transform(df_min_max[numeric_columns])

<ipython-input-31-091c912b04c0>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.26666667 0.86666667 0.66666667 ... 0.4        0.53333333 0.4       ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_min_max.loc[:, numeric_columns] = scaler.fit_transform(df_min_max[numeric_columns])


In [ ]:
df_min_max

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,PreInt_EduHx-Season,FGC-Season,Physical-Season,PreInt_EduHx-computerinternet_hoursday,Physical-Weight,Physical-Height,...,BIA-BIA_LST,BIA-BIA_LDM,BIA-BIA_ICW,BIA-BIA_Frame_num,BIA-BIA_Fat,BIA-BIA_FMI,BIA-BIA_FFMI,BIA-BIA_BMR,BIA-BIA_ECW,BIA-BIA_DEE
3,00115b9f,Winter,0.266667,0,Winter,Summer,Summer,0.000000,0.429022,0.490716,...,0.383296,0.398119,0.425619,0.5,0.364807,0.440460,0.341423,0.358956,0.375436,0.424623
4,0016bb22,Spring,0.866667,1,Fall,Summer,Summer,0.000000,0.600421,0.490716,...,0.117494,0.199338,0.590622,0.0,0.204643,0.318094,0.167600,0.473020,0.264961,0.295217
8,0069fbed,Summer,0.666667,0,Summer,Spring,Spring,0.666667,0.294427,0.278515,...,0.138186,0.488514,0.244213,0.5,0.179706,0.308451,0.248389,0.729283,0.361890,0.164531
9,0083e397,Summer,0.933333,1,Summer,Summer,Summer,0.000000,0.579390,0.649867,...,0.330978,0.579124,0.199993,0.5,0.405281,0.336218,0.316479,0.271066,0.426932,0.284811
10,0087dd65,Spring,0.400000,1,Spring,Winter,Winter,0.000000,0.334385,0.702918,...,0.502768,0.210892,0.363046,0.0,0.229161,0.463383,0.143596,0.127423,0.039867,0.449622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,0.200000,0,Fall,Fall,Fall,0.666667,0.353312,0.397878,...,0.275742,0.364180,0.300133,0.0,0.303487,0.402998,0.235815,0.246043,0.247582,0.342955
3955,ff8a2de4,Fall,0.533333,0,Fall,Fall,Fall,0.333333,0.433228,0.583554,...,0.465669,0.440391,0.504961,0.0,0.258079,0.290390,0.339683,0.447552,0.483284,0.488697
3957,ffcd4dbd,Fall,0.400000,0,Fall,Winter,Winter,0.000000,0.577287,0.596817,...,0.522801,0.558705,0.521263,0.5,0.588592,0.629650,0.455921,0.502576,0.511758,0.465862
3958,ffed1dd5,Spring,0.533333,0,Spring,Spring,Winter,0.333333,0.457413,0.880637,...,0.689191,0.687739,0.718922,0.5,0.000000,0.045412,0.183764,0.691156,0.680414,0.947506


# One Hot encoding

In [98]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype


columns_df = df_impute_random.columns[1:]
categorical_columns = []
for column in columns_df:
  if is_string_dtype(df_impute_random[column]):
  #  print(column)
   # print(df[column].unique())
    categorical_columns.append(column)


# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df_impute_random[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded,
                          columns=encoder.get_feature_names_out(categorical_columns))

print(one_hot_df.shape)
print(df_impute_random.shape)

df_impute_random.reset_index(drop=True, inplace=True)
one_hot_df.reset_index(drop=True, inplace=True)
df_sklearn_encoded = pd.concat([df_impute_random.drop(categorical_columns, axis=1), one_hot_df], axis=1)



print(df_sklearn_encoded.shape)

#df_sklearn_encoded

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


df_sklearn_encoded.head()

(2236, 32)
(2236, 69)
(2236, 93)


,id,Basic_Demos-Age,Basic_Demos-Sex,PreInt_EduHx-computerinternet_hoursday,Physical-Weight,Physical-Height,Physical-BMI,Physical-HeartRate,Physical-Diastolic_BP,Physical-Systolic_BP,PCIAT-PCIAT_Total,sii,PCIAT-PCIAT_11,PCIAT-PCIAT_02,PCIAT-PCIAT_01,PCIAT-PCIAT_10,PCIAT-PCIAT_20,PCIAT-PCIAT_06,PCIAT-PCIAT_14,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_12,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_15,PCIAT-PCIAT_19,PCIAT-PCIAT_05,PCIAT-PCIAT_07,PCIAT-PCIAT_13,PCIAT-PCIAT_18,PCIAT-PCIAT_16,PCIAT-PCIAT_17,SDS-SDS_Total_Raw,SDS-SDS_Total_T,CGAS-CGAS_Score,FGC-FGC_TL,FGC-FGC_CU,FGC-FGC_PU,FGC-FGC_SRR,FGC-FGC_SRL,FGC-FGC_TL_Zone,FGC-FGC_CU_Zone,FGC-FGC_PU_Zone,FGC-FGC_SRR_Zone,FGC-FGC_SRL_Zone,BIA-BIA_TBW,BIA-BIA_FFM,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_SMM,BIA-BIA_LST,BIA-BIA_LDM,BIA-BIA_ICW,BIA-BIA_Frame_num,BIA-BIA_Fat,BIA-BIA_FMI,BIA-BIA_FFMI,BIA-BIA_BMR,BIA-BIA_ECW,BIA-BIA_DEE,Basic_Demos-Enroll_Season_Fall,Basic_Demos-Enroll_Season_Spring,Basic_Demos-Enroll_Season_Summer,Basic_Demos-Enroll_Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter,FGC-Season_Fall,FGC-Season_Spring,FGC-Season_Summer,FGC-Season_Winter,Physical-Season_Fall,Physical-Season_Spring,Physical-Season_Summer,Physical-Season_Winter,PCIAT-Season_Fall,PCIAT-Season_Spring,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,CGAS-Season_Fall,CGAS-Season_Spring,CGAS-Season_Summer,CGAS-Season_Winter,BIA-Season_Fall,BIA-Season_Spring,BIA-Season_Summer,BIA-Season_Winter
0,00115b9f,9,0,0.0,81.6,56.0,18.292347,97.0,60.0,117.0,44.0,1.0,3.0,2.0,4.0,2.0,1.0,1.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,4.0,5.0,0.0,3.0,3.0,3.0,4.0,31.0,45.0,71.0,7.0,18.0,5.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,45.9966,62.7757,3.0,3.84191,18.2943,26.4798,58.9338,16.7790,30.4041,2.0,18.82430,4.22033,14.0740,1131.430,15.59250,1923.44,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0016bb22,18,1,0.0,58.4,47.0,21.899516,118.0,65.0,107.0,0.0,0.0,2.0,0.0,4.0,2.0,1.0,1.0,0.0,3.0,0.0,0.0,2.0,1.0,4.0,1.0,1.0,0.0,0.0,2.0,0.0,3.0,29.0,58.0,75.0,12.0,0.0,10.0,10.0,10.0,1.0,1.0,0.0,1.0,1.0,38.0720,82.0602,4.0,5.31573,17.6237,25.6772,74.0944,16.3802,24.6740,2.0,24.84220,2.17012,13.3809,953.178,6.37437,2655.89,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0069fbed,15,0,2.0,132.2,44.0,16.560201,76.0,76.0,115.0,19.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,0.0,0.0,49.0,53.0,65.0,12.0,0.0,0.0,15.5,10.0,1.0,1.0,0.0,0.0,0.0,45.4306,59.8357,2.0,2.01781,23.5052,23.8034,40.1663,15.5909,25.3977,2.0,-5.90917,4.06521,14.7795,920.050,7.43282,1957.04,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0083e397,19,1,1.0,134.6,66.0,16.146408,75.0,64.0,98.0,12.0,0.0,0.0,2.0,2.0,1.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,2.0,2.0,0.0,2.0,27.0,59.0,50.0,10.0,10.0,0.0,10.0,11.0,1.0,1.0,1.0,0.0,1.0,42.2709,52.4305,2.0,4.45328,16.6825,26.7782,40.9378,15.3511,21.1202,1.0,5.76182,2.66389,14.1011,1282.910,6.56327,1554.21,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0087dd65,11,1,0.0,101.2,57.0,14.373669,88.0,72.0,101.0,7.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,60.0,42.0,61.0,7.0,0.0,1.0,5.0,9.0,1.0,0.0,0.0,1.0,1.0,34.1079,44.5290,3.0,2.97680,20.3814,17.9894,45.3493,11.0195,26.7165,2.0,13.50250,1.84164,13.8798,1041.230,10.62680,2201.87,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
